In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import zipfile as zf
files = zf.ZipFile("/content/drive/MyDrive/FBHM.zip", 'r')
files.extractall('memes')
files.close()

In [3]:
!pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.0 MB/s eta 0:00:00
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16424 sha256=8ba6581aeb19215509e1037018c8aa1a0e62004671222d

In [4]:
import os
import json
import numpy as np
import torch
import torch.nn as nn

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms
from transformers import LxmertModel, LxmertTokenizer
from efficientnet_pytorch import EfficientNet
from sklearn.metrics import precision_score, recall_score, f1_score
from tqdm import tqdm
from PIL import Image
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
import os

class ToxicMemeDataset(Dataset):
    def __init__(self, data_list, img_dir, tokenizer, max_len, transform=None):
        self.data = data_list
        self.img_dir = img_dir
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.transform = transform

    def __len__(self):
        return len(self.data)  # Return the length of the data list

    def __getitem__(self, idx):
        item = self.data[idx]
        text = item["text"]
        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )

        # Ensure correct image path handling
        img_path = os.path.join(self.img_dir, item["img"].lstrip("/"))  # Remove leading '/'

        # Check if image exists
        if not os.path.exists(img_path):
            raise FileNotFoundError(f"Image not found: {img_path}")

        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        return {
            "input_ids": encoding["input_ids"].squeeze(0).long(),
            "attention_mask": encoding["attention_mask"].squeeze(0).long(),
            "image": image,
            "label": torch.tensor(item["label"], dtype=torch.long),
        }

In [ ]:
class ToxicMemeClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.lxmert = LxmertModel.from_pretrained("unc-nlp/lxmert-base-uncased")

        # EfficientNet backbone with partial unfreezing
        self.cnn_backbone = EfficientNet.from_pretrained('efficientnet-b4')
        for param in self.cnn_backbone.parameters():
            param.requires_grad = False
        for block in self.cnn_backbone._blocks[-6:]:
            for param in block.parameters():
                param.requires_grad = True

        self.visual_fc = nn.Linear(self.cnn_backbone._fc.in_features, 2048)
        self.cnn_backbone._fc = nn.Identity()

        # Enhanced classifier head
        self.classifier = nn.Sequential(
            nn.Linear(self.lxmert.config.hidden_size, 512),
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, input_ids, attention_mask, images):
        # Ensure image tensor is float
        images = images.float()

        visual_feats = self.cnn_backbone(images)
        visual_feats = self.visual_fc(visual_feats).unsqueeze(1)

        batch_size = visual_feats.size(0)
        visual_pos = torch.tensor([[0, 0, 1, 1]], device=device).repeat(batch_size, 1, 1).float()

        outputs = self.lxmert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            visual_feats=visual_feats,
            visual_pos=visual_pos,
        )
        return self.classifier(outputs.pooled_output)

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.8, gamma=2):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        ce_loss = nn.CrossEntropyLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-ce_loss)
        return (self.alpha * (1 - pt) ** self.gamma * ce_loss).mean()

def train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs):
    model = model.to(device).float()
    scheduler = ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.2)
    best_f1 = 0

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        progress = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")

        for batch in progress:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            images = batch["image"].to(device)
            labels = batch["label"].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, images=images)
            loss = criterion(outputs, labels)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            total_loss += loss.item()
            progress.set_postfix(loss=loss.item())

        val_metrics = evaluate_model(model, val_loader, device)
        scheduler.step(val_metrics['loss'])

        if val_metrics['f1'] > best_f1:
            best_f1 = val_metrics['f1']
            torch.save(model.state_dict(), "best_model.pth")
            print(f"New best model saved with F1: {best_f1:.4f}")

In [ ]:
def evaluate_model(model, data_loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    total_loss = 0
    criterion = nn.CrossEntropyLoss()

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            images = batch["image"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, images=images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return {
        'loss': total_loss / len(data_loader),
        'accuracy': (np.array(all_preds) == np.array(all_labels)).mean(),
        'precision': precision_score(all_labels, all_preds, zero_division=0),
        'recall': recall_score(all_labels, all_preds, zero_division=0),
        'f1': f1_score(all_labels, all_preds, zero_division=0)
    }

In [7]:
# Main Execution
if __name__ == "__main__":
    # Configuration
    BATCH_SIZE = 32
    MAX_LEN = 128
    LR = 2e-5
    EPOCHS = 5

    # Data Preparation
    with open("/content/train_balanced.json", "r") as f:
        all_data = json.load(f)

    # Split dataset
    train_size = int(0.8 * len(all_data))
    train_data, val_data = torch.utils.data.random_split(all_data, [train_size, len(all_data)-train_size])

    # Transforms with augmentation
    train_transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(0.2, 0.2, 0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    val_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    tokenizer = LxmertTokenizer.from_pretrained("unc-nlp/lxmert-base-uncased")

    train_dataset = ToxicMemeDataset(
        [all_data[i] for i in train_data.indices],
        "/content/memes/FBHM/data/img",
        tokenizer,
        MAX_LEN,
        train_transform
    )

    val_dataset = ToxicMemeDataset(
        [all_data[i] for i in val_data.indices],
        "/content/memes/FBHM/data/img",
        tokenizer,
        MAX_LEN,
        val_transform
    )

    # Handle class imbalance
    labels = [d["label"] for d in train_dataset.data]
    class_counts = np.bincount(labels)
    class_weights = 1 / np.sqrt(class_counts)
    samples_weights = class_weights[labels]
    sampler = WeightedRandomSampler(samples_weights, len(samples_weights))

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=sampler)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

    # Initialize model and training components
    model = ToxicMemeClassifier()
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=0.01)
    criterion = FocalLoss(alpha=0.75, gamma=1.5)

    # Train and evaluate
    train_model(model, train_loader, val_loader, optimizer, criterion, device, EPOCHS)
    model.load_state_dict(torch.load("best_model.pth"))
    final_metrics = evaluate_model(model, val_loader, device)

    print("\nFinal Validation Metrics:")
    print(f"Accuracy: {final_metrics['accuracy']:.4f}")
    print(f"Precision: {final_metrics['precision']:.4f}")
    print(f"Recall: {final_metrics['recall']:.4f}")
    print(f"F1 Score: {final_metrics['f1']:.4f}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/153 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Loaded pretrained weights for efficientnet-b4



Epoch 1/5: 100%|██████████| 134/134 [01:44<00:00,  1.28it/s, loss=0.146]


New best model saved with F1: 0.6439


Epoch 5/5: 100%|██████████| 134/134 [01:40<00:00,  1.33it/s, loss=0.0771]
<ipython-input-7-8cf2478716c8>:229: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_


Final Validation Metrics:
Accuracy: 0.6293
Precision: 0.6411
Recall: 0.6468
F1 Score: 0.6439
